# LangChain

In the previous lab, the `azure-cosmos` library was used to perform a vector search through a db command to find product documents that were most similar to the user's input. LangChain has a vector store class named [**AzureCosmosDBNoSqlVectorSearch**](https://github.com/langchain-ai/langchain/blob/master/docs/docs/integrations/vectorstores/azure_cosmos_db_no_sql.ipynb), that supports vector search in Azure Cosmos DB for NoSQL. However, at the time of this writing, due to the pace of LangChain development, the current implementation has a bug that impacts the retrieval of search results. As such, the code in this lab will create a LangChain [retriever class](https://python.langchain.com/docs/integrations/retrievers/) to connect to and search the vector store using the `azure-cosmos` library. More on retrievers in a moment.

In [1]:
import os
import json
import time
from pydantic import BaseModel
from typing import Type, TypeVar, List
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from azure.cosmos import CosmosClient, ContainerProxy
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import (
    AsyncCallbackManagerForRetrieverRun,
    CallbackManagerForRetrieverRun,
)
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import StructuredTool
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_functions_agent
from models import Product, SalesOrder

T = TypeVar('T', bound=BaseModel)

In [2]:
# Load settings for the notebook
load_dotenv()
CONNECTION_STRING = os.environ.get("COSMOS_DB_CONNECTION_STRING")
EMBEDDINGS_DEPLOYMENT_NAME = "embeddings"
COMPLETIONS_DEPLOYMENT_NAME = "completions"
AOAI_ENDPOINT = os.environ.get("AOAI_ENDPOINT")
AOAI_KEY = os.environ.get("AOAI_KEY")
AOAI_API_VERSION = "2024-06-01"

In [3]:
# Establish Azure OpenAI connectivity
llm = AzureChatOpenAI(            
        temperature = 0,
        openai_api_version = AOAI_API_VERSION,
        azure_endpoint = AOAI_ENDPOINT,
        openai_api_key = AOAI_KEY,         
        azure_deployment = "completions"
)
embedding_model = AzureOpenAIEmbeddings(
    openai_api_version = AOAI_API_VERSION,
    azure_endpoint = AOAI_ENDPOINT,
    openai_api_key = AOAI_KEY,   
    azure_deployment = "embeddings",
    chunk_size=800
)

# Initialize the Azure Cosmos DB client, database and product (with vector) container
client = CosmosClient.from_connection_string(CONNECTION_STRING)
db = client.get_database_client("cosmic_works")
product_v_container = db.get_container_client("product_v")
sales_order_container = db.get_container_client("salesOrder")

## RAG with LangChain

Recall that in previous labs the `products_v` container was created with an indexing policy and vector embedding policy to enable vector search. Each item in the container contains a contentVector field that contains the vectorized embeddings of the document itself.

In this section, we'll implement the RAG pattern using LangChain. In LangChain, a **retriever** is used to augment the prompt with contextual data. In this case, a custom LangChain retriever is needed. The return value of the invokation of retriever in LangChain is a list of `Document` objects. The LangChain `Document` class contains two properties: `page_content`, that represents the textual content that is typically used to augment the prompt, and `metadata` that contains all other attributes of the document. In this case, we'll use the document content as the `page_content` and include the similarity score as the metadata.

We'll also define a reusable RAG [chain](https://python.langchain.com/docs/modules/chains/) to control the flow and behavior of the call into the LLM. This chain is defined using the LCEL syntax (LangChain Expression Language).

In [4]:
class AzureCosmosDBNoSQLRetriever(BaseRetriever):
    """
    A custom LangChain retriever that uses Azure Cosmos DB NoSQL database for vector search.
    """
    embedding_model: AzureOpenAIEmbeddings
    container: ContainerProxy
    model: Type[T]
    vector_field_name: str
    num_results: int=5

    def __get_embeddings(self, text: str) -> List[float]:       
        """
        Returns embeddings vector for a given text.
        """
        embedding = self.embedding_model.embed_query(text)        
        time.sleep(0.5) # rest period to avoid rate limiting on AOAI
        return embedding
    
    def __get_item_by_id(self, id) -> T:
        """
        Retrieves a single item from the Azure Cosmos DB NoSQL database by its ID.
        """
        query = "SELECT * FROM itm WHERE itm.id = @id"
        parameters = [
            {"name": "@id", "value": id}
        ]    
        item = list(self.container.query_items(
            query=query,
            parameters=parameters,
            enable_cross_partition_query=True
        ))[0]
        return self.model(**item)
    
    def __delete_attribute_by_alias(self, instance: BaseModel, alias):
        for model_field in instance.model_fields:
            field = instance.model_fields[model_field]            
            if field.alias == alias:
                delattr(instance, model_field)
                return
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """
        Performs a synchronous vector search on the Azure Cosmos DB NoSQL database.
        """
        embedding = self.__get_embeddings(query)
        items = self.container.query_items(
            query=f"""SELECT TOP @num_results itm.id, VectorDistance(itm.{self.vector_field_name}, @embedding) AS SimilarityScore 
                    FROM itm
                    ORDER BY VectorDistance(itm.{self.vector_field_name}, @embedding)
                    """,
            parameters = [
                { "name": "@num_results", "value": self.num_results },
                { "name": "@embedding", "value": embedding }            
            ],
            enable_cross_partition_query=True
        ) 
        returned_docs = []
        for item in items:
            itm = self.__get_item_by_id(item["id"])  
            # Remove the vector field from the returned item so it doesn't fill the context window
            self.__delete_attribute_by_alias(itm, self.vector_field_name)            
            returned_docs.append(Document(page_content=json.dumps(itm, indent=4, default=str), metadata={"similarity_score": item["SimilarityScore"]}))
        return returned_docs
    
    async def _aget_relevant_documents(
        self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun
    ) -> List[Document]:
        """
        Performs an asynchronous vector search on the Azure Cosmos DB NoSQL database.        
        """
        raise Exception(f"Asynchronous search not implemented.")

In [5]:
products_retriever = AzureCosmosDBNoSQLRetriever(
    embedding_model = embedding_model,
    container = product_v_container,
    model = Product,
    vector_field_name = "contentVector",
    num_results = 5   
)

In [ ]:
query = "What yellow products are there?"
products_retriever.invoke(query)

In [7]:
# A system prompt describes the responsibilities, instructions, and persona of the AI.
# Note the addition of the templated variable/placeholder for the list of products and the incoming question.
system_prompt = """
Your name is "Willie". You are an AI assistant for the Cosmic Works bike store. You help people find production information for bikes and accessories. Your demeanor is friendly, playful with lots of energy.

Do not include citations or citation numbers in your responses. Do not include emojis.

Only answer questions related to the information provided in the list of products below that are represented
in JSON format.

If you are asked a question that is not in the list, respond with "I don't know."

Only answer questions related to Cosmic Works products, customers, and sales orders.

If a question is not related to Cosmic Works products, customers, or sales orders,
respond with "I only answer questions about Cosmic Works"

List of products:
{products}

Question:
{question}
"""

In [8]:
# Create the prompt template from the system_prompt text
llm_prompt = PromptTemplate.from_template(system_prompt)

rag_chain = (
    # populate the tokens/placeholders in the llm_prompt    
    # question is a passthrough that takes the incoming question
    { "products": products_retriever, "question": RunnablePassthrough()}
    | llm_prompt
    # pass the populated prompt to the language model
    | llm
    # return the string ouptut from the language model
    | StrOutputParser()
)

In [ ]:
question = "What are the names and skus of yellow products? Output the answer as a bulleted list."
response = rag_chain.invoke(question)
print(response)

## LangChain Agent

Remember, the concept of an agent is quite similar to that of a chain in LangChain but with one fundamental difference. A chain in LangChain is a hard-coded sequence of steps executed in a specific order. Conversely, an agent leverages the LLM to assess the incoming request with the current context to decide what steps or actions need to be executed and in what order.

LangChain agents can leverage tools and toolkits. A tool can be an integration into an external system, custom code, a retriever, or even another chain. A toolkit is a collection of tools that can be used to solve a specific problem.

 ### Create Agent Tools
 
 LangChain does have a built-in [`create_retriever_tool`](https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents#retriever-tool) that wraps a vector store retriever.

In [10]:
# Create a tool that will use the product vector search in Azure Cosmos DB for NoSQL
products_retriever_tool = create_retriever_tool(
    retriever = products_retriever,
    name = "vector_search_products",
    description = "Searches Cosmic Works product information for similar products based on the question. Returns the product information in JSON format."
)
tools = [products_retriever_tool]

### Tools part 2

Certain properties do not have semantic meaning (such as the GUID id field) and attempting to use vector search on these fields will not yield meaningful results. We need a tool to retrieve specific documents based on popular searches criteria.

The following tool definitions is not an exhaustive list of what may be needed but serves as an example to provide concrete lookups of a document in the Cosmic Works database.

In [11]:
# Tools helper methods
def delete_attribute_by_alias(instance: BaseModel, alias:str):
    for model_field in instance.model_fields:
        field = instance.model_fields[model_field]            
        if field.alias == alias:
            delattr(instance, model_field)
            return

def get_single_item_by_field_name(
        container:ContainerProxy, 
        field_name:str, 
        field_value:str, 
        model:Type[T]) -> T:
    """
    Retrieves a single item from the Azure Cosmos DB NoSQL database by a specific field and value.
    """
    query = f"SELECT TOP 1 * FROM itm WHERE itm.{field_name} = @value"
    parameters = [
        {
            "name": "@value", 
            "value": field_value
        }
    ]    
    item = list(container.query_items(
        query=query,
        parameters=parameters,
        enable_cross_partition_query=True
    ))[0]
    item_casted = model(**item)    
    return item_casted

In [12]:
def get_product_by_id(product_id: str) -> str:
    """
    Retrieves a product by its ID.    
    """
    item = get_single_item_by_field_name(product_v_container, "id", product_id, Product)
    delete_attribute_by_alias(item, "contentVector")
    return json.dumps(item, indent=4, default=str)    

def get_product_by_sku(sku: str) -> str:
    """
    Retrieves a product by its sku.
    """
    item = get_single_item_by_field_name(product_v_container, "sku", sku, Product)
    delete_attribute_by_alias(item, "contentVector")
    return json.dumps(item, indent=4, default=str)
    
def get_sales_by_id(sales_id: str) -> str:
    """
    Retrieves a sales order by its ID.
    """
    item = get_single_item_by_field_name(sales_order_container, "id", sales_id, SalesOrder)
    return json.dumps(item, indent=4, default=str)

tools.extend([
    StructuredTool.from_function(get_product_by_id),
    StructuredTool.from_function(get_product_by_sku),
    StructuredTool.from_function(get_sales_by_id)
])

### Create the agent

The [`create_openai_functions_agent`](https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents#agent-constructor) is a built-in agent that includes conversational history, tools selection, and agent scratchpad (for keeping track of the state of the progress of the LLM interaction).

Remember that an agent leverages the LLM to assess the incoming request with the current context to decide what steps or actions need to be executed and in what order. LangChain agents can leverage tools. A tool can be an integration into an external system, custom code, or even another chain.

In [13]:
agent_instructions = """           
        Your name is "Willie". You are an AI assistant for the Cosmic Works bike store. You help people find production information for bikes and accessories. Your demeanor is friendly, playful with lots of energy.
        Do not include citations or citation numbers in your responses. Do not include emojis.
        You are designed to answer questions about the products that Cosmic Works sells, the customers that buy them, and the sales orders that are placed by customers.
        If you don't know the answer to a question, respond with "I don't know."      
        Only answer questions related to Cosmic Works products, customers, and sales orders.
        If a question is not related to Cosmic Works products, customers, or sales orders,
        respond with "I only answer questions about Cosmic Works"
    """  

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", agent_instructions),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)  
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

**Note**: On the following agent_executor invocations it is safe to ignore the error: `Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")` - this is a defect in the verbose debug output of LangChain and does not affect the outcome of the invocation.

In [ ]:
result = agent_executor.invoke({"input": "What products do you have that are yellow?"})
print("***********************************************************")
print(result['output'])

In [ ]:
result = agent_executor.invoke({"input": "What products were purchased for sales order '06FE91D2-B350-471A-AD29-906BF4EB97C4' ?"})
print("***********************************************************")
print(result['output'])

In [ ]:
result = agent_executor.invoke({"input": "What was the sales order total for sales order '93436616-4C8A-407D-9FDA-908707EFA2C5' ?"})
print("***********************************************************")
print(result['output'])

In [ ]:
result = agent_executor.invoke({"input": "What was the price of the product with sku `FR-R92B-58` ?"})
print("***********************************************************")
print(result['output'])